# Million Song Dataset

This is a subset of the Million Song Dataset http://millionsongdataset.com/

This subset was taked from the UCI Machine Learning Repository https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD#

This dataset contains over 500,000 songs with the year of the song with 90 attributes relating to the timbre average and timbre covaraiance of the song.

We will create a model that can predict the year of the song based on the timbre attributes of the song.

---

First, we need to start Dask

This will start Dask using all the cores and memory on this compute node

In [1]:
import dask
import dask.distributed
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

/u/home/c/ccp2287/.conda/envs/mydask/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34410 instead
  warnings.warn(


<Client: 'tcp://127.0.0.1:33413' processes=6 threads=36, memory=187.60 GiB>

In [2]:
import socket
print(socket.gethostname())

n6466


---

Loading the dataset

We will create a Dask DataFrame (Pandas-like) from loading a csv file

In [3]:
import dask.dataframe as dd
path = "YearPredictionMSD.txt"
MSD_dd = dd.read_csv(path, header=None)
MSD_dd

Dask DataFrame Structure:
                  0        1        2        3        4        5        6        7        8        9        10       11       12       13       14       15       16       17       18       19       20       21       22       23       24       25       26       27       28       29       30       31       32       33       34       35       36       37       38       39       40       41       42       43       44       45       46       47       48       49       50       51       52       53       54       55       56       57       58       59       60       61       62       63       64       65       66       67       68       69       70       71       72       73       74       75       76       77       78       79       80       81       82       83       84       85       86       87       88       89       90
npartitions=7                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
               int64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64
                 ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...
...              ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...

Note that the MSD_dd Dask DateFrame does not have the values (yet)

Dask will read the csv file when it need to compute anything

In [4]:
MSD_dd.head()

0         1         2         3         4         5         6         7   \
0  2001  49.94357  21.47114  73.07750   8.74861 -17.40628 -13.09905 -25.01202   
1  2001  48.73215  18.42930  70.32679  12.94636 -10.32437 -24.83777   8.76630   
2  2001  50.95714  31.85602  55.81851  13.41693  -6.57898 -18.54940  -3.27872   
3  2001  48.24750  -1.89837  36.29772   2.58776   0.97170 -26.21683   5.05097   
4  2001  50.97020  42.20998  67.09964   8.46791 -15.85279 -16.81409 -12.48207   

         8         9   ...        81         82        83        84        85  \
0 -12.23257   7.83089  ...  13.01620  -54.40548  58.99367  15.37344   1.11144   
1  -0.92019  18.76548  ...   5.66812  -19.68073  33.04964  42.87836  -9.90378   
2  -2.35035  16.07017  ...   3.03800   26.05866 -50.92779  10.93792  -0.07568   
3 -10.34124   3.55005  ...  34.57337 -171.70734 -16.96705 -46.67617 -12.51516   
4  -9.37636  12.63699  ...   9.92661  -55.95724  64.92712 -17.72522  -1.49237   

         86         87        88         89        90  
0 -23.08793   68.40795  -1.82223  -27.46348   2.26327  
1 -32.22788   70.49388  12.04941   58.43453  26.92061  
2  43.20130 -115.00698  -0.05859   39.67068  -0.66345  
3  82.58061  -72.08993   9.90558  199.62971  18.85382  
4  -7.50035   51.76631   7.88713   55.66926  28.74903  

[5 rows x 91 columns]

---

Now, lets create some DataFrames to prepare for Machine Learning!

The first column MSD_dd[0] is the year of the song (y_data). 

The rest of the columns are the timbre attributes of the song (X_data)

We are trying to predict the song year based on the timbre attributes of the song

In [5]:
X_data = MSD_dd.iloc[:, MSD_dd.columns != 0]
X_data

Dask DataFrame Structure:
                    1        2        3        4        5        6        7        8        9        10       11       12       13       14       15       16       17       18       19       20       21       22       23       24       25       26       27       28       29       30       31       32       33       34       35       36       37       38       39       40       41       42       43       44       45       46       47       48       49       50       51       52       53       54       55       56       57       58       59       60       61       62       63       64       65       66       67       68       69       70       71       72       73       74       75       76       77       78       79       80       81       82       83       84       85       86       87       88       89       90
npartitions=7                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
               float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64
                   ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...
...                ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ..

In [6]:
y_data = MSD_dd.iloc[:, MSD_dd.columns == 0]
y_data

Dask DataFrame Structure:
                   0
npartitions=7       
               int64
                 ...
...              ...
                 ...
                 ...
Dask Name: getitem, 2 graph layers

---

Next, we will using train_test_split from the dask_ml package to split our data.

We will create Training and Test (Numpy-like) Dask arrays 

Note the "chunks" need to be of similar size

In [7]:
from dask_ml.model_selection import train_test_split
import dask.array as da

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33)

y_train = y_train.to_dask_array(lengths=True).rechunk((50)).ravel()
y_test = y_test.to_dask_array(lengths=True).rechunk((50)).ravel()

X_train = X_train.to_dask_array(lengths=True).rechunk((50,50))
X_test = X_test.to_dask_array(lengths=True).rechunk((50,50))

/u/home/c/ccp2287/.conda/envs/mydask/lib/python3.10/site-packages/dask_ml/model_selection/_split.py:462: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


In [8]:
y_train

dask.array<reshape, shape=(345285,), dtype=int64, chunksize=(50,), chunktype=numpy.ndarray>

In [9]:
X_train

dask.array<rechunk-merge, shape=(345285, 90), dtype=float64, chunksize=(50, 50), chunktype=numpy.ndarray>

---

We will use a Random Forest Model from scikit-learn

In [10]:
from sklearn.ensemble import RandomForestRegressor
import joblib 

model = RandomForestRegressor()

from joblib import parallel_backend
with parallel_backend('dask'):
 model.fit(X_train,y_train)

---

Use our model to make predictions from the Test set

In [11]:
import numpy as np

with parallel_backend('dask'):
 predict_test = model.predict(X_test)
    
model_error = np.abs(y_test - predict_test)


In [12]:
from sklearn.metrics import mean_absolute_error

print("test MAE: ", mean_absolute_error(y_test, predict_test))

test MAE:  6.523720503547767
